<a href="https://colab.research.google.com/github/gsdos1984-sudo/EPPsimulatorby-Miguel-verastegui/blob/main/kidbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio==4.*


In [2]:
# KidBot – Asistente familiar para Iker & Itzel (con fallback sin Gradio)
# Autor: Miguel + ChatGPT (Viernes)
# ----------------------------------------------
# ✔ Corre aún si NO tienes gradio instalado (modo CLI)
# ✔ Si tienes gradio, levanta una app web local
# ✔ Incluye pruebas unitarias:  python kidbot.py --test
# ✔ Forzar CLI:                 python kidbot.py --cli
# ✔ Datos persistentes en:      kids_bot_data.json (o variable KIDBOT_DATA_FILE)
# ----------------------------------------------
# Requisitos (opcional para UI web):
#   pip install gradio==4.*
# Uso típico (UI web):
#   python kidbot.py
# Si falta Gradio o usas --cli, corre en modo consola con los mismos comandos básicos.

import os
import json
import random
import datetime
import argparse
import sys
from typing import Dict, Any, List, Tuple

# ----------------------------------------------
# Intento de importar Gradio (opcional)
# ----------------------------------------------
try:
    import gradio as gr  # type: ignore
    GRADIO_AVAILABLE = True
except Exception:
    gr = None  # type: ignore
    GRADIO_AVAILABLE = False

# ----------------------------------------------
# Configuración de almacenamiento
# Permite redirigir el archivo de datos para pruebas con la variable de entorno
# ----------------------------------------------
DATA_FILE = os.environ.get("KIDBOT_DATA_FILE", "kids_bot_data.json")
CHILDREN = ["Iker", "Itzel"]
DEFAULT_GOALS = {"chips_goal_week": 30}
DEFAULT_ROUTINE = {
    "morning": [
        "Levantarse 6:00 am",
        "Tender la cama",
        "Cepillarse dientes",
        "Ponerse uniforme",
        "Preparar mochila y lonche",
    ],
    "evening": [
        "Dejar mochila lista",
        "Bañarse",
        "Cenar saludable",
        "Apagar pantallas 9:00 pm",
        "Dormir 10:00 pm",
    ],
}

STARTER_PHRASES = [
    "¿Listo para practicar 5 frases en inglés?",
    "¿Quieres un cuento corto personalizado?",
    "¿Revisamos tu rutina y sumamos fichas?",
    "¿Hacemos 5 operaciones de mates súper rápidas?",
]

ENGLISH_CARDS = [
    ("How's it going?", "¿Cómo va todo?"),
    ("Can you help me, please?", "¿Me puedes ayudar, por favor?"),
    ("I don’t understand, could you repeat?", "No entiendo, ¿puedes repetir?"),
    ("What time is it?", "¿Qué hora es?"),
    ("I’m working on my homework.", "Estoy haciendo la tarea."),
]

MATH_LEVELS = {
    "Iker": {"range": (0, 12), "ops": ["+", "-", "*"]},
    "Itzel": {"range": (0, 10), "ops": ["+", "-"]},
}

STORY_TEMPLATES = [
    "Había una vez un/a {hero} que vivía en {place}. Un día, encontró {object} y decidió {quest}. En el camino, aprendió {lesson}. Al final, {ending}.",
    "Un {hero} de {place} quería {quest}. Con la ayuda de {friend}, venció {challenge} y descubrió {lesson}. Desde entonces, {ending}.",
]

STORY_BANK = {
    "hero": ["explorador", "robot amable", "gatita valiente", "inventor", "astronauta"],
    "place": ["un bosque mágico", "Opelika", "una nave espacial", "un taller secreto", "un parque con columpios"],
    "object": ["un mapa brillante", "unas gafas superinteligentes", "una mochila musical", "un libro misterioso"],
    "quest": ["ayudar a un amigo", "crear un invento", "aprender inglés", "salvar un árbol", "organizar su cuarto"],
    "friend": ["un perrito curioso", "una luciérnaga", "un vecino nuevo", "un robotito"],
    "challenge": ["el miedo a la oscuridad", "un rompecabezas difícil", "una tormenta", "el desorden"],
    "lesson": ["que pedir ayuda está bien", "que practicar cada día funciona", "que compartir da felicidad", "que la calma es poderosa"],
    "ending": ["durmió feliz", "celebró con su familia", "agradeció y siguió aprendiendo", "sonrió porque lo logró"],
}

# ----------------------------------------------
# Persistencia
# ----------------------------------------------

def load_data() -> Dict[str, Any]:
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    data = {
        "kids": {name: {"chips": 0, "routine": DEFAULT_ROUTINE, "history": []} for name in CHILDREN},
        "goals": DEFAULT_GOALS,
        "week_start": datetime.date.today().isoformat(),
    }
    save_data(data)
    return data


def save_data(data: Dict[str, Any]) -> None:
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


DATA = load_data()

# ----------------------------------------------
# Lógica de fichas (chips)
# ----------------------------------------------

def add_chips(kid: str, amount: int, reason: str) -> str:
    if kid not in DATA["kids"]:
        raise ValueError(f"Niño desconocido: {kid}")
    DATA["kids"][kid]["chips"] += int(amount)
    DATA["kids"][kid]["history"].append({
        "ts": datetime.datetime.now().isoformat(timespec="seconds"),
        "delta": int(amount),
        "reason": reason,
    })
    save_data(DATA)
    return f"✅ {kid} recibió {int(amount)} fichas por: {reason}. Total actual: {DATA['kids'][kid]['chips']}"


def reset_week() -> str:
    for kid in CHILDREN:
        DATA["kids"][kid]["chips"] = 0
        DATA["kids"][kid]["history"].clear()
    DATA["week_start"] = datetime.date.today().isoformat()
    save_data(DATA)
    return "♻️ Reinicié las fichas de la semana. ¡A empezar con todo!"

# ----------------------------------------------
# Rutinas
# ----------------------------------------------

def routine_checklist(kid: str) -> str:
    if kid not in DATA["kids"]:
        raise ValueError(f"Niño desconocido: {kid}")
    r = DATA["kids"][kid]["routine"]
    out = [f"📋 **Rutina de {kid}**"]
    out.append("\n**Mañana**:")
    for i, item in enumerate(r["morning"], start=1):
        out.append(f"{i}. [ ] {item}")
    out.append("\n**Noche**:")
    for i, item in enumerate(r["evening"], start=1):
        out.append(f"{i}. [ ] {item}")
    out.append("\n*Sugerencia:* al completar toda la rutina del día, da 2 fichas.")
    return "\n".join(out)

# ----------------------------------------------
# Inglés
# ----------------------------------------------

def english_quiz(kid: str, n: int = 5) -> Tuple[str, List[Tuple[str, str]]]:
    if kid not in DATA["kids"]:
        raise ValueError(f"Niño desconocido: {kid}")
    n = max(1, min(n, len(ENGLISH_CARDS)))
    sample = random.sample(ENGLISH_CARDS, k=n)
    lines = [f"🗣️ {kid}, repite después de mí (lee en voz alta):"]
    for i, (en, es) in enumerate(sample, start=1):
        lines.append(f"{i}. {en}  —  *{es}*")
    lines.append("\nTip: Practica pronunciación: marca el acento y ritmo natural.")
    return "\n".join(lines), sample

# ----------------------------------------------
# Matemáticas
# ----------------------------------------------

def make_math_problem(kid: str) -> Tuple[str, int]:
    if kid not in MATH_LEVELS:
        raise ValueError(f"Niño desconocido: {kid}")
    rmin, rmax = MATH_LEVELS[kid]["range"]
    a, b = random.randint(rmin, rmax), random.randint(rmin, rmax)
    op = random.choice(MATH_LEVELS[kid]["ops"])
    expr = f"{a} {op} {b}"
    answer = eval(expr)
    return expr, answer

# ----------------------------------------------
# Cuentos
# ----------------------------------------------

def make_story(kid: str, topic: str = "") -> str:
    if kid not in DATA["kids"]:
        raise ValueError(f"Niño desconocido: {kid}")
    template = random.choice(STORY_TEMPLATES)
    values = {k: random.choice(v) for k, v in STORY_BANK.items()}
    if topic:
        values["quest"] = topic
    story = template.format(**values)
    return f"📖 Cuento para {kid}:\n\n" + story

# ----------------------------------------------
# Motor de Chat basado en reglas (compartido por UI y CLI)
# ----------------------------------------------

def bot_reply(kid: str, history: List[Tuple[str, str]], message: str) -> str:
    msg = (message or "").strip().lower()

    if "rutina" in msg or "routine" in msg:
        return routine_checklist(kid)
    if "cuento" in msg or "story" in msg:
        return make_story(kid)
    if "inglés" in msg or "english" in msg:
        text, _ = english_quiz(kid)
        return text
    if "math" in msg or "mates" in msg or "matem" in msg:
        expr, _ = make_math_problem(kid)
        return f"🧮 Resuelve: {expr} = ?  (Escribe tu respuesta)"

    if msg.isdigit():
        # Buscar último reto de mates en el historial y validar
        last = next((h for h in reversed(history) if "Resuelve:" in h[1]), None)
        if last:
            expr = last[1].split(":", 1)[1].split("=")[0].strip()
            try:
                correct = eval(expr)
                if int(msg) == correct:
                    add_chips(kid, 1, f"Resolver {expr}")
                    return "🎉 ¡Correcto! Ganaste 1 ficha. ¿Otra operación o quieres un cuento?"
                else:
                    return f"Casi, intenta de nuevo. Pista: {expr}"
            except Exception:
                pass

    return random.choice(STARTER_PHRASES)

# ----------------------------------------------
# UI Gradio (solo si está disponible)
# ----------------------------------------------

def launch_gradio_app() -> None:
    assert GRADIO_AVAILABLE and gr is not None, "Gradio no está disponible"

    def make_child_tab(kid: str):
        with gr.Tab(kid):
            gr.Markdown(
                f"## Hola, {kid}! Soy tu compañero de aprendizaje ✨\nEscribe 'rutina', 'english', 'mates', 'story' o hazme una pregunta."
            )
            gr.ChatInterface(
                fn=lambda m, h: bot_reply(kid, h, m),
                title=f"KidBot – {kid}",
                examples=["rutina", "english", "mates", "cuento"],
                cache_examples=False,
                textbox=gr.Textbox(placeholder="Escribe aquí...", scale=7),
            )
            with gr.Row():
                with gr.Column():
                    btn_quiz = gr.Button("Practicar 5 frases en inglés")
                    out_quiz = gr.Markdown()
                with gr.Column():
                    btn_story = gr.Button("Cuento sorpresa")
                    out_story = gr.Markdown()
            btn_quiz.click(lambda: english_quiz(kid)[0], outputs=out_quiz)
            btn_story.click(lambda: make_story(kid), outputs=out_story)

    def parent_dashboard():
        with gr.Tab("Padres"):
            gr.Markdown("# Panel para papás 👨‍👩‍👧‍👦\nAdministra fichas, rutinas y metas semanales.")
            with gr.Row():
                kid_sel = gr.Dropdown(choices=CHILDREN, value=CHILDREN[0], label="Niño")
                amount = gr.Number(value=1, precision=0, label="Fichas a agregar (+) o quitar (-)")
                reason = gr.Textbox(value="Buen comportamiento / rutina completa", label="Razón")
                btn_add = gr.Button("Actualizar fichas")
            msg = gr.Markdown()
            btn_add.click(lambda k, a, r: add_chips(k, int(a), r), inputs=[kid_sel, amount, reason], outputs=msg)

            gr.Markdown("## Progreso actual")
            with gr.Row():
                chips_iker = gr.Number(value=lambda: DATA['kids']['Iker']['chips'], label="Iker – fichas", interactive=False)
                chips_itzel = gr.Number(value=lambda: DATA['kids']['Itzel']['chips'], label="Itzel – fichas", interactive=False)
            refresh_btn = gr.Button("🔄 Refrescar")
            refresh_btn.click(lambda: (DATA['kids']['Iker']['chips'], DATA['kids']['Itzel']['chips']), outputs=[chips_iker, chips_itzel])

            with gr.Accordion("Metas y reinicio semanal", open=False):
                goal = gr.Number(value=lambda: DATA['goals']['chips_goal_week'], label="Meta de fichas por semana")
                set_goal = gr.Button("Guardar meta")
                reset_btn = gr.Button("Reiniciar semana")
                goal_msg = gr.Markdown()
                set_goal.click(lambda g: _set_goal(g), inputs=goal, outputs=goal_msg)
                reset_btn.click(lambda: reset_week(), outputs=goal_msg)

            with gr.Accordion("Rutinas", open=False):
                kid_r = gr.Dropdown(choices=CHILDREN, value=CHILDREN[0], label="Niño")
                r_morning = gr.Textbox(value="\n".join(DEFAULT_ROUTINE["morning"]), lines=6, label="Mañana (una por línea)")
                r_evening = gr.Textbox(value="\n".join(DEFAULT_ROUTINE["evening"]), lines=6, label="Noche (una por línea)")
                save_r = gr.Button("Guardar rutina")
                r_msg = gr.Markdown()
                save_r.click(lambda k, m, e: _save_routine(k, m, e), inputs=[kid_r, r_morning, r_evening], outputs=r_msg)

    with gr.Blocks(title="KidBot – Iker & Itzel") as demo:
        gr.Markdown("# KidBot – Iker & Itzel\nUn bot local, seguro y divertido para practicar inglés, mates, cuentos y rutinas.")
        for kid in CHILDREN:
            make_child_tab(kid)
        parent_dashboard()

    demo.launch(share=('google.colab' in sys.modules))

# ----------------------------------------------
# Acciones de administración (metas/rutinas)
# ----------------------------------------------

def _set_goal(val: float) -> str:
    DATA["goals"]["chips_goal_week"] = int(val)
    save_data(DATA)
    return f"🎯 Meta semanal actualizada a {int(val)} fichas."


def _save_routine(kid: str, morning: str, evening: str) -> str:
    if kid not in DATA["kids"]:
        raise ValueError(f"Niño desconocido: {kid}")
    DATA["kids"][kid]["routine"] = {
        "morning": [x.strip() for x in morning.splitlines() if x.strip()],
        "evening": [x.strip() for x in evening.splitlines() if x.strip()],
    }
    save_data(DATA)
    return f"✅ Rutina actualizada para {kid}."

# ----------------------------------------------
# CLI (fallback sin Gradio)
# ----------------------------------------------

def run_cli() -> None:
    print("""
==============================
 KidBot – Modo Consola (CLI)
==============================
Comandos:
  - rutina / english / mates / cuento
  - chips +N "razón"   (ej: chips +2 "rutina completa")
  - reset               (reinicia la semana)
  - cambiar <Iker|Itzel>
  - salir

Tip: Instala la UI web con:  pip install gradio==4.*
""")
    current_kid = CHILDREN[0]
    history: List[Tuple[str, str]] = []
    print(f"Usando niño actual: {current_kid}\n")

    while True:
        try:
            raw = input(f"[{current_kid}] > ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nAdiós 👋")
            break

        if not raw:
            continue
        if raw.lower() in ("salir", "exit", "quit"):
            print("Adiós 👋")
            break
        if raw.lower().startswith("cambiar"):
            parts = raw.split()
            if len(parts) >= 2 and parts[1] in CHILDREN:
                current_kid = parts[1]
                history.clear()
                print(f"Ahora hablando con {current_kid}.")
            else:
                print(f"Uso: cambiar <{'|'.join(CHILDREN)}> ")
            continue
        if raw.lower().startswith("chips"):
            # formato: chips +N "razón"
            try:
                # simple parse
                parts = raw.split()
                amt = int(parts[1])  # permite +N o -N
                reason = raw.split("\"", 2)
                reason = reason[1] if len(reason) > 1 else "Actualización manual"
                print(add_chips(current_kid, amt, reason))
            except Exception as e:
                print(f"No pude procesar: {e}. Ejemplo: chips +2 \"rutina completa\"")
            continue
        if raw.lower() == "reset":
            print(reset_week())
            continue

        # flujo normal
        bot_msg = bot_reply(current_kid, history, raw)
        history.append((raw, bot_msg))
        print(bot_msg)

# ----------------------------------------------
# TESTS (unittest)
# ----------------------------------------------

def _reset_state_for_tests(tmp_path: str) -> None:
    global DATA_FILE, DATA
    DATA_FILE = tmp_path
    if os.path.exists(tmp_path):
        try:
            os.remove(tmp_path)
        except Exception:
            pass
    DATA = load_data()


def run_tests() -> int:
    import unittest
    import tempfile

    class KidBotTests(unittest.TestCase):
        def setUp(self):
            random.seed(0)
            self.tmp = tempfile.NamedTemporaryFile(delete=False)
            self.tmp.close()
            _reset_state_for_tests(self.tmp.name)

        def tearDown(self):
            try:
                os.remove(self.tmp.name)
            except Exception:
                pass

        def test_load_and_defaults(self):
            self.assertIn("kids", DATA)
            self.assertIn("Iker", DATA["kids"])
            self.assertEqual(DATA["kids"]["Iker"]["chips"], 0)

        def test_add_chips(self):
            msg = add_chips("Iker", 2, "prueba")
            self.assertIn("recibió 2 fichas", msg)
            self.assertEqual(DATA["kids"]["Iker"]["chips"], 2)

        def test_routine_contains_sections(self):
            text = routine_checklist("Itzel")
            self.assertIn("**Mañana**", text)
            self.assertIn("**Noche**", text)

        def test_english_quiz_len(self):
            text, cards = english_quiz("Iker", n=3)
            self.assertEqual(len(cards), 3)
            self.assertIn("repite después de mí", text)

        def test_math_problem_consistency(self):
            expr, ans = make_math_problem("Itzel")
            self.assertEqual(ans, eval(expr))

        def test_bot_reply_math_flow(self):
            history: List[Tuple[str, str]] = []
            prompt = bot_reply("Iker", history, "mates")
            self.assertIn("Resuelve:", prompt)
            history.append(("mates", prompt))
            # extraer expresión
            expr = prompt.split(":", 1)[1].split("=")[0].strip()
            correct = str(eval(expr))
            ok = bot_reply("Iker", history, correct)
            self.assertIn("¡Correcto!", ok)

    suite = unittest.defaultTestLoader.loadTestsFromTestCase(KidBotTests)
    result = unittest.TextTestRunner(verbosity=2).run(suite)
    return 0 if result.wasSuccessful() else 1

# ----------------------------------------------
# Entry point
# ----------------------------------------------
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="KidBot – Iker & Itzel")
    parser.add_argument("--cli", action="store_true", help="Forzar modo consola incluso si hay Gradio")
    parser.add_argument("--test", action="store_true", help="Ejecutar pruebas unitarias y salir")
    # En Colab/Jupyter pueden llegar args desconocidos (-f ...). Ignóralos.
    args, _ = parser.parse_known_args()

    if args.test:
        sys.exit(run_tests())

    if args.cli or not GRADIO_AVAILABLE:
        if not GRADIO_AVAILABLE and not args.cli:
            print("⚠️  Gradio no está instalado: pip install gradio==4.*")
            print("    Iniciando modo consola (CLI) automáticamente...\n")
        run_cli()
    else:
        launch_gradio_app()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ee5a492e6241a8f0cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
